In [4]:
import requests
import re

In [5]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
wiki = requests.get(
    'https://en.wikipedia.org/wiki/List_of_one-hit_wonders_in_the_United_States', headers=headers)


In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki.text, 'html.parser')


In [7]:
import pandas as pd


In [8]:
songhtml = soup.find_all('li')[14:580]
songs = [dict(zip(['Artist', 'Song'], [eachItem.getText()
              for eachItem in eachSong.find_all('a')[:2]])) for eachSong in songhtml]


In [9]:
lyricSearch = requests.get(
    f"https://www.google.com/search?q={songs[35]['Artist']} {songs[35]['Song']} lyrics", headers=headers)

In [10]:
lyricSoup = BeautifulSoup(lyricSearch.text, 'html.parser')


In [23]:
a = ''.join([eachPart.getText() for eachPart in lyricSoup.find_all("div", {"class": "BNeawe tAd8D AP7Wnd"})])


In [31]:
songtest = songs[1:4].copy()

In [32]:
import time
for eachSong in songtest:
    lyricSearch = requests.get(
    f"https://www.google.com/search?q={eachSong['Artist']} {eachSong['Song']} lyrics", headers=headers)
    lyricSoup = BeautifulSoup(lyricSearch.text, 'html.parser')
    eachSong['Lyrics'] = ''.join([eachPart.getText() for eachPart in lyricSoup.find_all("div", {"class": "BNeawe tAd8D AP7Wnd"})][:-1])
    time.sleep(.8)

In [33]:
songtest

[{'Artist': 'The Crows',
  'Song': 'Gee',
  'Lyrics': "Do do-do do, do-do do, do-do  do-do-do\nDo do-do do, do-do do, do-do  do-do-do\nDo do-do do, do-do do, do-do  do-do-do do\nLove that girl\n\nOh Gee, my oh Gee, well o-o Gee\nWhy I love that girl, love that girl\n\nOh please, listen to me, hear-hear-hear my plea\nWhy I love that girl\n\nHold me, Baby, squeeze me\nNever let me goDo do-do do, do-do do, do-do  do-do-do\nDo do-do do, do-do do, do-do  do-do-do\nDo do-do do, do-do do, do-do  do-do-do do\nLove that girl\n\nOh Gee, my oh Gee, well o-o Gee\nWhy I love that girl, love that girl\n\nOh please, listen to me, hear-hear-hear my plea\nWhy I love that girl\n\nHold me, Baby, squeeze me\nNever let me goDo do-do do, do-do do, do-do  do-do-do\nDo do-do do, do-do do, do-do  do-do-do\nDo do-do do, do-do do, do-do  do-do-do do\nLove that girl\n\nOh Gee, my oh Gee, well o-o Gee\nWhy I love that girl, love that girl\n\nOh please, listen to me, hear-hear-hear my plea\nWhy I love that girl\n\n